In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/best-efficientnet-b0/pytorch/default/1/best_efficientnet_b0 (1).pt
/kaggle/input/resnet18-tl/pytorch/default/1/best_resnet18.pt
/kaggle/input/photoplethysmography-ppg-dataset/PPG_Dataset.csv


# 📘 **PPG → Wavelet → CNN (Transfer Learning) — Complete Pipeline Summary**

This document summarizes the entire end-to-end pipeline used for **PPG-based MI classification** using **Wavelet Transform + Transfer Learning CNNs**.

---

# 📌 **1. Preprocessing of PPG Signals**

### ✔ Each PPG sample is a 1-dimensional signal  
Example shape:
2000


### ✔ Convert 1D PPG → 2D Wavelet Scalogram (CWT)

We used:

- **Wavelet:** Morlet (`"morl"`)
- **Scales:** 1 to 127  
- **Output shape:**  
127 frequency bins × 2000 time points

  
### ✔ Why Wavelet Transform?

PPG is **non-stationary** → frequency changes with time.

Wavelet captures:

- Heartbeat morphology
- Dicrotic notch abnormalities
- HRV-related changes
- Low–high frequency bursts
- MI-induced waveform distortions

Wavelet images are **2D** → perfect for CNNs.

---

# 📌 **2. Converting PPG → Wavelet Image**

### Steps:

1. Compute **CWT**
2. Take **absolute magnitude**
3. Normalize → **0–255**
4. Convert to **PIL Image**
5. Resize to **224×224**
6. Convert to **3-channel**
7. Apply **ImageNet normalization**

This makes wavelet scalograms compatible with ImageNet pretrained CNNs.

---

# 📌 **3. Dataset Pipeline (On-the-Fly Wavelet Generation)**

We do **NOT** save scalograms on disk.

Instead:

- Compute the CWT **inside `__getitem__()`**
- Only the 1D signal is stored
- Saves >10GB RAM
- Very fast for Colab/Kaggle

This gives a clean, memory-efficient, GPU-friendly pipeline.

---

# 📌 **4. Transfer Learning — Why?**

PPG datasets are small → training CNN from scratch would **overfit**.

Transfer learning allows:

- Using ImageNet pretrained filters
- Learning generic edges + shapes
- Fine-tuning only a **small number of parameters**
- Faster convergence
- Better generalization

---

# 📌 **5. Architectures Used**

We trained **3 models**:

---

## 🟦 **A. ResNet-18 (Pretrained)**

### ✔ Architecture Highlights
- 18-layer residual network  
- Skip-connections  
- Robust for medical images  
- Light & fast  

### ✔ Fine-Tuning Strategy
- Freeze **all layers**
- Unfreeze **layer4** only
- Replace last FC:
fc → Linear(in_features, 1)


### ✔ Why it works?
- Learns MI-related:
- high-frequency bursts  
- low-frequency drops  
- morphological waveform abnormalities  

---

## 🟧 **B. MobileNetV2 (Lightweight)**

### ✔ Architecture Highlights
- Depthwise separable convolution  
- Only **3.4M parameters**  
- Best for **edge devices**  

### ✔ Fine-Tuning
- Freeze all  
- Unfreeze **features[-1]**  
- Replace classifier head  

---

## 🟩 **C. EfficientNet-B0 (Best Accuracy)**

### ✔ Architecture Highlights
- Depth × Width × Resolution scaling  
- Squeeze-and-Excitation  
- Very efficient  

### ✔ Fine-Tuning
- Freeze all
- Unfreeze **features[-1]**
- Replace classifier:

classifier[1] = Linear(in_features, 1)


### ✔ Why it’s best?
- Captures **subtle MI patterns**:
- refined time-frequency interactions  
- subtle heartbeat irregularities  
- small morphological distortions  

---

# 📌 **6. Output Layer & Loss**

### ✔ Output Activation: **Sigmoid**  
### ✔ Loss: **BCEWithLogitsLoss**

Why?

- Stable gradient behavior
- Perfect for binary classification (MI vs Normal)
- Avoids numerical instability of manual sigmoid + BCE

---

# 📌 **7. Training Strategy**

- Train **only last conv block**
- Optimizer: **Adam**
- Learning Rate: **1e-4**
- Batch size: **16**
- Epochs: **5–10**
- Use **DataLoader(num_workers=2)**

---

# 📌 **8. Results (Accuracy)**

| Model            | Accuracy |
|------------------|----------|
| ResNet-18        |   96%    |
| MobileNetV2      |   94%    |
| EfficientNet-B0  |   95%    |

**ResNET18 performed best.**

---

# 📌 **9. Why This Approach Works So Well**

✔ Wavelet transform reveals MI-specific frequency patterns  
✔ CNN captures complex time-frequency structures  
✔ Transfer learning prevents overfitting  
✔ Lightweight + accurate  
✔ Real-time capable  

Perfect for:

- Wearable health devices  
- Clinical triage  
- Remote heart monitoring  
- Cardiovascular diagnostics  

---

# 📌 **10. Summary of Techniques Used**

✔ Wavelet Transform (CWT)  
✔ Image normalization & resizing  
✔ On-the-fly wavelet generation  
✔ Transfer Learning:  
- ResNet-18  
- MobileNetV2  
- EfficientNet-B0  

✔ Freeze all → Unfreeze last block  
✔ BCEWithLogitsLoss  
✔ Adam optimizer  
✔ Accuracy evaluation  


In [2]:
import torch
import torchvision.models as models
import torch.nn as nn

MODEL_PATH = "/kaggle/input/resnet18-tl/pytorch/default/1/best_resnet18.pt"

# Build same architecture
model = models.resnet18(weights=None)
model.fc = nn.Linear(model.fc.in_features, 1)

# Load weights
model.load_state_dict(torch.load(MODEL_PATH, map_location="cpu"))

model.eval()
print("Model loaded successfully!")


Model loaded successfully!


In [3]:
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
import pywt
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

class PPGWaveletDataset(Dataset):
    def __init__(self, df, scales=np.arange(1,128), wavelet='morl'):
        self.X = df.drop("Label", axis=1).values
        self.y = df["Label"].values
        self.scales = scales
        self.wavelet = wavelet

        self.transform = T.Compose([
            T.ToPILImage(),
            T.Resize((224,224)),
            T.Grayscale(num_output_channels=3),
            T.ToTensor(),
            T.Normalize(mean=[0.485,0.456,0.406],
                        std=[0.229,0.224,0.225])
        ])

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        signal = self.X[idx]

        coeffs, freqs = pywt.cwt(signal, self.scales, self.wavelet)
        scalogram = np.abs(coeffs)

        img_norm = (scalogram - scalogram.min()) / (scalogram.max() - scalogram.min() + 1e-12)
        img_uint8 = (img_norm * 255).astype(np.uint8)

        img = self.transform(img_uint8)
        label = float(self.y[idx])
        return img, torch.tensor(label, dtype=torch.float32)


In [5]:
df = pd.read_csv("/kaggle/input/photoplethysmography-ppg-dataset/PPG_Dataset.csv")

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df['Label'] = le.fit_transform(df['Label'])


In [6]:
from torch.utils.data import random_split

dataset = PPGWaveletDataset(df)

val_size = int(0.2 * len(dataset))
train_size = len(dataset) - val_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

val_loader = DataLoader(val_ds, batch_size=16, shuffle=False)


In [7]:
from sklearn.metrics import accuracy_score

def evaluate(model, loader):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs  # CPU inference
            outputs = model(imgs)

            # Convert output → probability
            probs = torch.sigmoid(outputs).numpy().flatten()
            preds = (probs >= 0.5).astype(int)

            y_true.extend(labels.numpy().astype(int))
            y_pred.extend(preds)

    return accuracy_score(y_true, y_pred)

acc = evaluate(model, val_loader)
print("Validation Accuracy:", acc)


Validation Accuracy: 0.9592233009708738
